In [1]:
from bs4 import BeautifulSoup
import re

import pandas as pd
import numpy as np
from urllib.request import urlopen
from selenium import webdriver

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import tkinter as tk




In [2]:
try:
    links = pd.read_csv('links_worked.csv')
except:
    links = pd.read_csv('links.csv')
    links.insert(2, "Info", None)
links.head()


,Company Name,Website,Info
0,Nestle,www.nestle.com,None
1,Dr. Reddy's Laboratories,www.drreddys.com,None
2,Coca-Cola,www.coca-colacompany.com,None
3,Pfizer,www.pfizer.com,None
4,PepsiCo,www.pepsico.com,None


In [3]:
def get_screen_size():
    root = tk.Tk()
    root.withdraw()  # Hide the root window
    width = root.winfo_screenwidth()
    height = root.winfo_screenheight()
    return width, height

# Example usage
width, height = get_screen_size()
print(f"Screen width: {width}, Screen height: {height}")

Screen width: 1920, Screen height: 1080


In [4]:
def headless_selenium_crawler(url):
   chrome_options = webdriver.ChromeOptions()
   # set a headless driver
   chrome_options.add_argument('--headless') 
   # set the user-agent back to chrome.
   user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'
   chrome_options.add_argument(f'user-agent={user_agent}')
   driver = webdriver.Chrome(options=chrome_options)
   driver.set_window_size(height, width) # set the size of the window
   driver.get(url)
   page_text = driver.page_source.lower()
   driver.quit()
   return page_text

   


In [5]:
def selenium_crawler(url):
    driver = webdriver.Chrome()
    driver.get(url)
    page_text = driver.page_source.lower()
    driver.quit()
    return page_text



In [6]:
def get_text_from_html(html):
    soup = BeautifulSoup(html, features="html.parser")

    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()    # rip it out

    # get text
    text = soup.get_text()

    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = '\n'.join(chunk for chunk in chunks if chunk)
    return text



In [7]:
def check_validity(text):
    # Convert text to string to handle NaN or float values
    text = str(text)
    print('>>> Checked Validitystart:',text[:10])
    if 'just a moment...' in text:
        return None
    elif '404' in text:
        return None
    else:
        text = text.replace('\n', ' ')
        text = text.replace(',', ' ')
        # removing all the symbotls and numbers using re
        text = re.sub(r'[^A-Za-z\s]', '', text)
        return text



In [8]:
def get_company_info(url):
    try:
        text=headless_selenium_crawler(url)
        text=get_text_from_html(text)
    except Exception as e:
        print('>>> Request failed from headless_selenium_crawler:', e)
        text=selenium_crawler(url)
        text=get_text_from_html(text)
    except:
        print('>>> Request failed in get_company_info:', e)
        return "No info found"
    return text
    
    

In [9]:
def save_company_info():
    for i in range(len(links)):
    #for i in range(10):
        if links.loc[i, "Info"] == None or links.loc[i, "Info"] == '' or links.loc[i, "Info"] == 'No info found' or links.loc[i, "Info"] == np.nan :
            company_name = links.iloc[i, 0]
            print(f'>>> {i} Company Name:', company_name)
            website = links.iloc[i, 1]
            try:
                website = 'https://' + website
                info = get_company_info(website)
            except Exception as e:
                info = 'No info found'
                print('>>> Failed in save_company_info try:', e)
            try:
                links.loc[i, "Info"] = info
            except:
                links.insert(2, "Info", None)
                links.loc[i, "Info"] = info
    
        
        links.loc[i, "Info"] = check_validity(links.loc[i, "Info"])
            

save_company_info()
links.to_csv('links_worked.csv', index=False, header=True)



>>> 0 Company Name: Nestle
>>> Checked Validitystart: just a mom
>>> 1 Company Name: Dr. Reddy's Laboratories
>>> Checked Validitystart: dr. reddy’
>>> 2 Company Name: Coca-Cola
>>> Checked Validitystart: the coca-c
>>> 3 Company Name: Pfizer
>>> Checked Validitystart: pfizer: on
>>> 4 Company Name: PepsiCo
>>> Checked Validitystart: home page

>>> 5 Company Name: Johnson & Johnson
>>> Checked Validitystart: johnson & 
>>> 6 Company Name: Danone
>>> Checked Validitystart: world food
>>> 7 Company Name: Bayer
>>> Checked Validitystart: bayer – gl
>>> 8 Company Name: General Mills
>>> Checked Validitystart: general mi
>>> 9 Company Name: GlaxoSmithKline (GSK)
>>> Checked Validitystart: home | gsk
>>> 10 Company Name: Kellogg’s
>>> Checked Validitystart: unleashing
>>> 11 Company Name: Merck & Co.
>>> Checked Validitystart: merck | ho
>>> 12 Company Name: Unilever
>>> Checked Validitystart: unilever g
>>> 13 Company Name: Roche
>>> Checked Validitystart: roche - do
>>> 14 Company Name: Ne

In [10]:
links

,Company Name,Website,Info
0,Nestle,www.nestle.com,None
1,Dr. Reddy's Laboratories,www.drreddys.com,dr reddys laboratories good health cant waity...
2,Coca-Cola,www.coca-colacompany.com,the cocacola company refresh the world make a ...
3,Pfizer,www.pfizer.com,pfizer one of the worlds premier biopharmaceut...
4,PepsiCo,www.pepsico.com,home page your privacy is important to us this...
5,Johnson & Johnson,www.jnj.com,johnson johnson changing health for humanity ...
6,Danone,www.danone.com,world food company danone skip to content abo...
7,Bayer,www.bayer.com,bayer global home bayer global sorry you ne...
8,General Mills,www.generalmills.com,general mills a us based food company general...
9,GlaxoSmithKline (GSK),www.gsk.com,home gsk skip to content skip to search globa...


In [11]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

def preprocess_text(text):
    # Tokenization
    tokens = word_tokenize(text, preserve_line=True)
    
    # Stopword Removal
    stop_words = set(stopwords.words())
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in filtered_tokens]
    
    return lemmatized_tokens


In [12]:
try:
    links.insert(len(links.columns), "Processed_Info", None)
except:
    links.drop(columns=["Processed_Info"], inplace=True)
    links.insert(len(links.columns), "Processed_Info", None)

for i in range(len(links)):
    info = links.loc[i, "Info"]
    processed_text = preprocess_text(str(info))
    # Ensure processed_text is a single value
    links.loc[i, "Processed_Info"] = processed_text if isinstance(processed_text, str) else str(processed_text)


In [13]:
links.head()

,Company Name,Website,Info,Processed_Info
0,Nestle,www.nestle.com,None,[]
1,Dr. Reddy's Laboratories,www.drreddys.com,dr reddys laboratories good health cant waity...,"['dr', 'reddys', 'laboratory', 'health', 'wait..."
2,Coca-Cola,www.coca-colacompany.com,the cocacola company refresh the world make a ...,"['cocacola', 'company', 'refresh', 'world', 'm..."
3,Pfizer,www.pfizer.com,pfizer one of the worlds premier biopharmaceut...,"['pfizer', 'world', 'premier', 'biopharmaceuti..."
4,PepsiCo,www.pepsico.com,home page your privacy is important to us this...,"['home', 'page', 'privacy', 'important', 'webs..."


In [34]:
def categorize_company(text, categories):
    category_scores = {category: 0 for category in categories}
    
    for category, keywords in categories.items():
        for keyword in keywords:
            if str(keyword).lower() in str(text).lower():
                category_scores[category] += 1
    
    # Determine the category with the highest score
    assigned_category = max(category_scores, key=category_scores.get)
    return assigned_category



In [35]:
bulk_yes = [
    "bulk", "manufacturing", "production capacity", "industrial scale", 
    "large-scale production", "bioprocessing", "fermentation", "strains", 
    "raw materials", "custom blends", "supply", "biotechnology", 
    "contract manufacturing", "OEM", "CMO", "bulk supply"
]

bulk_no = [
    "retail", "consumer products", "end-user", "finished goods", 
    "direct to consumer", "D2C", "B2C", "brand", "marketing", 
    "sales", "distribution", "retail outlets", "e-commerce", 
    "online store", "consumer-facing"
]



In [36]:
# Define categories and keywords
categories_bulk = {
    "Bulk": bulk_yes,
    "Retail": bulk_no
}


for i in range(len(links)):
    category = categorize_company(links.loc[i, "Processed_Info"], categories_bulk)
    links.loc[i, "Bulk/Retail"] = category

In [37]:
brand_yes = [
    "brand", "consumer products", "retail", "end-user", "finished goods", 
    "direct to consumer", "D2C", "B2C", "marketing", "sales", 
    "distribution", "retail outlets", "e-commerce", "online store", 
    "consumer-facing", "product line", "brand identity", "brand recognition"
]

brand_no = [
    "bulk", "manufacturing", "production capacity", "industrial scale", 
    "large-scale production", "bioprocessing", "fermentation", "strains", 
    "raw materials", "custom blends", "supply", "biotechnology", 
    "contract manufacturing", "OEM", "CMO", "bulk supply"
]

In [38]:
# Define categories and keywords
categories_brand = {
    "Brand": brand_yes,
    "Non-Brand": brand_no
}


for i in range(len(links)):
    category = categorize_company(links.loc[i, "Processed_Info"], categories_brand)
    links.loc[i, "Brand/Non-Brand"] = category

In [39]:
Fb = [
    "bacillus coagulans", "fortification", "milk", "cereal", "bakery", "dairy",
    "beverages", "nutrition", "health drinks", "functional foods", "probiotic foods",
    "yogurt", "cheese", "fermented foods", "snacks", "energy bars", "vitamin-enriched"
]

Formulations = [
    "bacipro", "provinorm", "cognisol", "product", "description", "formulation",
    "finished products", "pharmaceuticals", "nutraceuticals", "capsules", "tablets",
    "sachets", "oral suspension", "clinical trials", "regulatory compliance", "R&D"
]

Health_Segments = [
    "gut", "women's", "cognitive", "health", "mental wellness", "sports nutrition",
    "digestive health", "immune support", "well-being", "brain health",
    "stress relief", "hormonal balance", "memory enhancement", "focus", "endurance"
]

Distributors = [
    "raw", "materials", "nutraceuticals", "probiotics", "supply chain", "wholesale",
    "distribution", "ingredients", "sourcing", "logistics",
    "network", "partnerships", "global reach", "inventory", "market trends"
]

In [40]:
# Define categories and keywords
categories_main = {
    "F&B": Fb,
    "Formulations": Formulations,
    "Health Segments": Health_Segments,
    "Distributors": Distributors
}


for i in range(len(links)):
    category = categorize_company(links.loc[i, "Processed_Info"], categories_main)
    links.loc[i, "Category"] = category


In [41]:
probiotics = [
    'probiotics', 'probiotic', 'lactobacillus', 'bifidobacterium', 
    'gut flora', 'microbiome', 'fermented', 'live cultures', 
    'beneficial bacteria', 'gut bacteria', 'prebiotics', 'synbiotics'
]

fortification = [
    'fortification', 'fortified', 'enriched', 'enhanced', 
    'added nutrients', 'vitamin fortified', 'mineral fortified', 
    'nutrient-dense', 'supplemented', 'nutrient enrichment', 
    'vitamin-enriched', 'mineral-enriched'
]

In [42]:


# Keywords to search for
categories_type = {
    'probiotics': probiotics,
    'fortification': fortification
}

for i in range(len(links)):
    category = categorize_company(links.loc[i, "Processed_Info"], categories_type)
    links.loc[i, "Category_Type"] = category

In [43]:
gut_health = [
    'gut health', 'digestive health', 'constipation', 'diarrhea',
    'intestinal health', 'bowel health', 'digestive system', 
    'bloating', 'indigestion', 'IBS', 'irritable bowel syndrome', 
    'gut microbiota', 'stomach health', 'probiotic support'
]

womens_health = [
    'women\'s health', 'UTI', 'PCOD', 'PCOS', 'menstrual health', 
    'hormonal balance', 'fertility', 'pregnancy health', 
    'menopause', 'breast health', 'reproductive health', 
    'gynecological health', 'ovarian health'
]

cognitive_health = [
    'cognitive health', 'mental wellness', 'anxiety', 'brain health', 
    'memory', 'focus', 'concentration', 'mental clarity', 
    'neuroprotection', 'cognitive function', 'stress relief', 
    'depression', 'mood enhancement', 'mental fatigue'
]

In [44]:
    
# Keywords to search for
categories_health = {
    'gut_health': gut_health,
    'womens_health': womens_health,
    'cognitive_health': cognitive_health,
}

for i in range(len(links)):
    category = categorize_company(links.loc[i, "Processed_Info"], categories_health)
    links.loc[i, "Category_Health"] = category

links.head()

,Company Name,Website,Info,Processed_Info,Category,Category_Type,Category_Health,Bulk/Retail,Brand/Non-Brand,Relevance
0,Nestle,www.nestle.com,None,[],F&B,probiotics,gut_health,Bulk,Brand,Not Relevant
1,Dr. Reddy's Laboratories,www.drreddys.com,dr reddys laboratories good health cant waity...,"['dr', 'reddys', 'laboratory', 'health', 'wait...",Distributors,probiotics,womens_health,Bulk,Brand,Relevant
2,Coca-Cola,www.coca-colacompany.com,the cocacola company refresh the world make a ...,"['cocacola', 'company', 'refresh', 'world', 'm...",F&B,probiotics,womens_health,Retail,Brand,Relevant
3,Pfizer,www.pfizer.com,pfizer one of the worlds premier biopharmaceut...,"['pfizer', 'world', 'premier', 'biopharmaceuti...",Health Segments,fortification,womens_health,Bulk,Non-Brand,Relevant
4,PepsiCo,www.pepsico.com,home page your privacy is important to us this...,"['home', 'page', 'privacy', 'important', 'webs...",F&B,probiotics,gut_health,Retail,Brand,Relevant


In [45]:
links.head()

,Company Name,Website,Info,Processed_Info,Category,Category_Type,Category_Health,Bulk/Retail,Brand/Non-Brand,Relevance
0,Nestle,www.nestle.com,None,[],F&B,probiotics,gut_health,Bulk,Brand,Not Relevant
1,Dr. Reddy's Laboratories,www.drreddys.com,dr reddys laboratories good health cant waity...,"['dr', 'reddys', 'laboratory', 'health', 'wait...",Distributors,probiotics,womens_health,Bulk,Brand,Relevant
2,Coca-Cola,www.coca-colacompany.com,the cocacola company refresh the world make a ...,"['cocacola', 'company', 'refresh', 'world', 'm...",F&B,probiotics,womens_health,Retail,Brand,Relevant
3,Pfizer,www.pfizer.com,pfizer one of the worlds premier biopharmaceut...,"['pfizer', 'world', 'premier', 'biopharmaceuti...",Health Segments,fortification,womens_health,Bulk,Non-Brand,Relevant
4,PepsiCo,www.pepsico.com,home page your privacy is important to us this...,"['home', 'page', 'privacy', 'important', 'webs...",F&B,probiotics,gut_health,Retail,Brand,Relevant


In [46]:
# Define the relevancy function
def determine_relevance(row):
    if (row['Info'] == None or row['Info'] == '' or row['Info'] == 'No info found' or row['Info'] == np.nan):
        return 'Not Relevant'
    elif (row['Bulk/Retail'] == 'Bulk' and row['Category_Health'] in ['gut_health', 'womens_health', 'cognitive_health']) or \
       (row['Brand/Non-Brand'] == 'Brand' and row['Category_Type'] in ['probiotics', 'fortification']):
        return 'Relevant'
    else:
        return 'Not Relevant'

for i in range(len(links)):
    relevance = determine_relevance(links.loc[i])
    links.loc[i, "Relevance"] = relevance

links.head()

,Company Name,Website,Info,Processed_Info,Category,Category_Type,Category_Health,Bulk/Retail,Brand/Non-Brand,Relevance
0,Nestle,www.nestle.com,None,[],F&B,probiotics,gut_health,Bulk,Brand,Not Relevant
1,Dr. Reddy's Laboratories,www.drreddys.com,dr reddys laboratories good health cant waity...,"['dr', 'reddys', 'laboratory', 'health', 'wait...",Distributors,probiotics,womens_health,Bulk,Brand,Relevant
2,Coca-Cola,www.coca-colacompany.com,the cocacola company refresh the world make a ...,"['cocacola', 'company', 'refresh', 'world', 'm...",F&B,probiotics,womens_health,Retail,Brand,Relevant
3,Pfizer,www.pfizer.com,pfizer one of the worlds premier biopharmaceut...,"['pfizer', 'world', 'premier', 'biopharmaceuti...",Health Segments,fortification,womens_health,Bulk,Non-Brand,Relevant
4,PepsiCo,www.pepsico.com,home page your privacy is important to us this...,"['home', 'page', 'privacy', 'important', 'webs...",F&B,probiotics,gut_health,Retail,Brand,Relevant


In [47]:
links.Relevance.value_counts()


Relevance
Relevant        37
Not Relevant     3
Name: count, dtype: int64

In [48]:
links_final = links[['Company Name', 'Website', 'Bulk/Retail', 'Brand/Non-Brand', 'Category', 'Category_Type', 'Category_Health', 'Relevance']]
links_final.to_csv('links_final.csv', index=False, header=True)
